In [51]:
using Distributions
using Optim

In [52]:
delta1, delta2 = -3 , -2
beta1, beta2 = [1 4], [2 3]
rho = -2
lambda = exp(rho)/(1+exp(rho))

N = 1000
Y = zeros(N,2)
X = zeros(N,2)

io = open("monte_carlo.csv", "w")
println(io, "y1,y2,x1,x2")

for i=1:N
    x1,x2 = rand(Normal(0.0, 1.0),3)
    e1,e2 = rand(Logistic(),2)
    select = rand(Categorical([lambda, (1-lambda)]))
    
    #see logit response i.e. definitely 11
    eq = []
    for y_init=[[0 0],[1 1],[0 1],[1 0]]
        y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0
        
        if false
            println( "$(1*y1) $(1*y2),$(y_init)" )
            println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
            println("-----")
        end
            
        if (y1==y_init[1]) & (y2==y_init[2])
            append!(eq,[y_init])
        end
    end
    #println( eq )
    #println( "========")
    y1, y2 = eq[1]
    if size(eq)[1] >= 2
        y1, y2 = eq[select]
    end
    Y[i,:] =[y1 y2]
    X[i,:] =[x1 x2]
    println(io, "$(y1),$(y2),$(x1),$(x2)")
end

close(io)

In [53]:
function ll_obj(params;xs=0,ys=0)
    n = size(xs)[1]
    k = Integer(size(X)[2]/2) + 1
    p = zeros(2,2,n)

    #solve for the conditional probability firm i does not enter
    #index 1- firm, index2 - firm -i's action
    for a_j=0:1
        util1 = params[1] .+ xs[:,1].*params[2]  .+ a_j.*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ a_j.*params[6]
        p[1,a_j+1,:] = 1 ./(1 .+ exp.(util1))
        p[2,a_j+1,:] = 1 ./(1 .+ exp.(util2))
    end
    
    lambda = exp(params[7])/(1+exp(params[7]))
 
    #solve for probablity of nash
    mult_eq = max.(( p[1,2,:]  .- p[1,1,:] ) .* (  p[2,2,:] .- p[2,1,:] ),0)
    prob01 = ( p[1,2,:] ) .* ( 1 .- p[2,1,:] ) - (1-lambda).*mult_eq
    prob10 = ( 1 .- p[1,1,:] ) .* ( p[2,2,:] ) - (lambda).*mult_eq
    prob00 = p[1,1,:] .* p[2,1,:]
    prob11 = ( 1 .- p[1,2,:] ) .* ( 1 .- p[2,2,:] )
    
    #compute empirical likelihood
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    ll = p00 .* prob00 + p11 .*prob11 .+ p01 .* prob01 .+ p10 .* prob10

    return -sum(log.(max.(ll,0)))    
end

ll_obj([1 4 -3 2 3 -2 -2];xs=X,ys=Y)

619.5547948089882

In [54]:
a = 10
b =(exp(a)/(1+exp(a)))^2 + 1/4 + 1/2*exp(a)/(1+exp(a))
println( (exp(a)/(1+exp(a)))^2 )
println( 1/2*exp(a)/(1+exp(a)) )
print(b)

0.9999092063235617
0.4999773010656488
1.7498865073892105

In [55]:
function maximum_likelihood(params;xs=0,ys=0)
    ll_trial(params) = ll_obj(params;xs=xs,ys=ys)
    return optimize(ll_trial, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -2.];xs=X,ys=Y)
print(result.minimizer)
print(result.minimum)

[0.969911 4.02015 -2.66917 1.93822 3.00727 -1.82395 -24.6406]615.987924116095